<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_12thApril2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.csv  ": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "ST_GR=(\"G1\")   ST in (\"10,19,20,21\")",
        "ST_GR=(\"G2\")   ST in (\"11,12,13,14,15,16,17,18\")",
        "ST_GR=(\"G3\")   Otherwise",
        "POP=(1)",
        "WRK=(1)  CWS in (\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=(0)  CWS NOT in (\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=(1)      CWS in (\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=(0)      Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
        "E 100*LF/POP",
        "R LF/POP

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        pass

def GROUP_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')
            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

def FILTER_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('FILTER').lower() in str(key).lower():
            ### Replace logical AND '&&' with 'and'
            if '&&' in value[0]:
                return value[0].replace('&&', '&')
            ### Replace logical OR '||' with 'or'
            elif '||' in value[0]:
                return value[0].replace('||', '|')
            else:
                return value[0]

def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)

                else:
                    col_name = col_name.upper()
                    print(type(col_name))


def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 0


def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value
                
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                 
                    if match:
                        condition = match.group(1)
                        
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)
                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        

def EST_RSE_(data):
    for key, value in data.items():
        pass

TRANSFORM_(data)[2]
RENAME_(data, df)
NEWVARIABLE_(data, df)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [6]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'df["\1"]==', query_string)


    # Define a function to replace variable names with df["<variable_name>"]
    def replace_var(match):
        return f'df["{match.group(0)}"]'

    # Use regex to replace variable names with df["<variable_name>"]
    pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)
    
    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

# new_lst = [s.replace("['", "[").replace("']", "]") for s in final_parsed_query]
# print([s.replace("['", "[").replace("']", "]") for s in final_parsed_query if s else pass])
# final_parsed_query = [s.replace("['", "[").replace("']", "]") if s is not None else None for s in final_parsed_query]
final_parsed_query

["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [7]:
df.head()

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,GEDU_LVL,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,1.0,51,51,51,0,0,01280101,0,0,0
1,1,28,17469,1,1,1,1,1,1,1,...,8.0,91,91,91,0,0,01280101,0,0,0
2,1,28,17469,1,1,1,1,1,1,1,...,10.0,91,91,91,0,0,01280101,0,0,0
3,1,28,17469,1,1,1,1,1,1,1,...,1.0,51,51,51,0,0,01280101,0,0,0
4,1,28,17469,2,1,1,2,1,1,1,...,5.0,51,51,51,0,0,01280101,0,0,0


In [8]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'STRMID',
       'POP', 'LF', 'WRK'],
      dtype='object')

In [9]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

print(Arr1)
print(Arr2)
print(Arr3)

['ST_GR', 'ST_GR', 'ST_GR', 'POP', 'WRK', 'WRK', 'LF', 'LF']
['G1', 'G2', 'G3', '1', '1', '0', '1', '0']
["ST.isin(['10,19,20,21'])", "ST.isin(['11,12,13,14,15,16,17,18'])", 'Otherwise', None, "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])", "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])", "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])", 'Otherwise']


In [10]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,GEDU_LVL,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,1.0,51,51,51,0,0,01280101,0,0,0
1,1,28,17469,1,1,1,1,1,1,1,...,8.0,91,91,91,0,0,01280101,0,0,0
2,1,28,17469,1,1,1,1,1,1,1,...,10.0,91,91,91,0,0,01280101,0,0,0


In [11]:
set(Arr1) - set(list(df.columns))

{'ST_GR'}

In [12]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

{'ST_GR'}

In [13]:
Arr3

["ST.isin(['10,19,20,21'])",
 "ST.isin(['11,12,13,14,15,16,17,18'])",
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [14]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 'Otherwise',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 'Otherwise']

In [15]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set([Arr3[j] for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = '~' + ' & ~'.join(other_values)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)

new_arr3

['ST.isin([10,19,20,21])',
 'ST.isin([11,12,13,14,15,16,17,18])',
 '~ST.isin([10,19,20,21]) & ~ST.isin([11,12,13,14,15,16,17,18])',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72,81])']

In [16]:
Arr3 = new_arr3

In [17]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):
    if query_str is True:
        df[Arr1[i]] = Arr2[i]
    else:
        df_query = df.query(query_str)
        df.loc[df_query.index, Arr1[i]] = Arr2[i]

print(df)

        SEC  ST    FSU  IDNO  SEG  SSS  SSU  STRM  SSTRM  SS  ...  PAS  PS_SS  \
0         1  28  17469     1    1    1    1     1      1   1  ...   51     51   
1         1  28  17469     1    1    1    1     1      1   1  ...   91     91   
2         1  28  17469     1    1    1    1     1      1   1  ...   91     91   
3         1  28  17469     1    1    1    1     1      1   1  ...   51     51   
4         1  28  17469     2    1    1    2     1      1   1  ...   51     51   
...     ...  ..    ...   ...  ...  ...  ...   ...    ...  ..  ...  ...    ...   
109851    2  34  27428     7    1    4    1     1      3   2  ...   94     94   
109852    2  34  27428     8    1    4    2     1      3   2  ...   51     51   
109853    2  34  27428     8    1    4    2     1      3   2  ...   91     91   
109854    2  34  27428     8    1    4    2     1      3   2  ...   92     92   
109855    2  34  27428     8    1    4    2     1      3   2  ...   94     94   

        CWS  ERN_SELF  ERN_

In [18]:
df['ST_GR'].unique()

array(['G3', 'G2', 'G1'], dtype=object)

In [19]:
df['POP'].unique()

array(['1'], dtype=object)

In [20]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method.csv')

In [21]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'STRMID',
       'POP', 'LF', 'WRK', 'ST_GR'],
      dtype='object')

In [22]:
import pandas as pd
import numpy as np

# # Read the data from the CSV file
# df = pd.read_csv('Work_File1.csv')

# Get the current column names
old_col_names = df.columns.tolist()

# Create a dictionary with the new column names
new_col_names = {col: col.strip() for col in old_col_names}

# Rename the columns using the dictionary
df.rename(columns=new_col_names, inplace=True)

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df.apply(lambda row: row['MULT'] / 200 if row['NSC'] > row['NSS'] else row['MULT'] / 100, axis=1)

# Group the data by SEC and ST_GR and calculate the aggregates
grouped_df = df.groupby(['SEC', 'ST_GR']).agg({'POP': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                'LF': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                'WRK': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                'MULT': 'count'})



# Rename the columns and reset the index
grouped_df = grouped_df.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat', 'MULT': 'no_sam'}).reset_index()

# Save the results to a new CSV file
grouped_df.to_csv('W1.csv', index=False)


In [23]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,CWS,ERN_SELF,ERN_REG,STRMID,POP,LF,WRK,ST_GR,SS_MULT,CMULT
0,1,28,17469,1,1,1,1,1,1,1,...,51,0,0,01280101,1,1,1,G3,25102.96,12551.48
1,1,28,17469,1,1,1,1,1,1,1,...,91,0,0,01280101,1,0,0,G3,25102.96,12551.48
2,1,28,17469,1,1,1,1,1,1,1,...,91,0,0,01280101,1,0,0,G3,25102.96,12551.48


In [27]:
df['SS_MULT']

0         25102.96
1         25102.96
2         25102.96
3         25102.96
4         25102.96
            ...   
109851     1961.08
109852     1961.08
109853     1961.08
109854     1961.08
109855     1961.08
Name: SS_MULT, Length: 109856, dtype: float64

In [28]:
# Filter dataframe to include only rows where SS is 1 or 2
df = df.query("SS in [1, 2]")
print(f"Number of rows after filtering: {len(df)}")

# Generate stratum and stratum x sub-sample level estimates
w2 = df.groupby(['SEC', 'ST_GR', 'STRMID']).agg(
    popstr=('POP', 'sum'),
    ## 'POP': lambda x: (pd.to_numeric(x) * df['CMULT']).sum()
    s1pop=('POP', lambda x: np.sum(np.where(df['SS'] == 1, pd.to_numeric(x) * df['SS_MULT'], 0))),
    s2pop=('POP', lambda x: np.sum(np.where(df['SS'] == 2, pd.to_numeric(x) * df['SS_MULT'], 0))),
    lfstr=('LF', 'sum'),
    s1lf=('LF', lambda x: np.sum(np.where(df['SS'] == 1, pd.to_numeric(x) * df['SS_MULT'], 0))),
    s2lf=('LF', lambda x: np.sum(np.where(df['SS'] == 2, pd.to_numeric(x) * df['SS_MULT'], 0))),
    wrkstr=('WRK', 'sum'),
    s1wrk=('WRK', lambda x: np.sum(np.where(df['SS'] == 1, pd.to_numeric(x) * df['SS_MULT'], 0))),
    s2wrk=('WRK', lambda x: np.sum(np.where(df['SS'] == 2, pd.to_numeric(x) * df['SS_MULT'], 0)))
).reset_index()


Number of rows after filtering: 109856


In [33]:
# Join w2 and w1 on SEC and ST_GR columns
w3 = w2.merge(grouped_df, on=['SEC', 'ST_GR'], suffixes=('_str', '_grp'))

# Add new columns to w3
w3['r1hat'] = w3['lfhat'] / w3['pophat']
w3['r2hat'] = w3['wrkhat'] / w3['pophat']

In [34]:
# Equate s1pop and s2pop
w3.loc[w3['popstr'] == w3['s1pop'], 's1pop'] = w3.loc[w3['popstr'] == w3['s1pop'], 's2pop']
w3.loc[w3['popstr'] == w3['s2pop'], 's2pop'] = w3.loc[w3['popstr'] == w3['s2pop'], 's1pop']

# Equate s1lf and s2lf
w3.loc[w3['lfstr'] == w3['s1lf'], 's1lf'] = w3.loc[w3['lfstr'] == w3['s1lf'], 's2lf']
w3.loc[w3['lfstr'] == w3['s2lf'], 's2lf'] = w3.loc[w3['lfstr'] == w3['s2lf'], 's1lf']

# Equate s1wrk and s2wrk
w3.loc[w3['wrkstr'] == w3['s1wrk'], 's1wrk'] = w3.loc[w3['wrkstr'] == w3['s1wrk'], 's2wrk']
w3.loc[w3['wrkstr'] == w3['s2wrk'], 's2wrk'] = w3.loc[w3['wrkstr'] == w3['s2wrk'], 's1wrk']

In [35]:
w4 = w3.groupby(['SEC', 'ST_GR', 'no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat']).agg(
    var_pop=('s1pop', lambda x: ((x - x.mean()) ** 2).sum()),
    mse_R1=('s1lf', lambda x: ((x - x.mean()) ** 2).sum() + (x.name[-2] ** 2) * (((x.name[3] - x.name[3].mean()) ** 2).sum()) - 
           2 * x.name[-2] * ((x - x.mean()) * (x.name[3] - x.name[3].mean())).sum()),
    mse_R2=('s1wrk', lambda x: ((x - x.mean()) ** 2).sum() + (x.name[-1] ** 2) * (((x.name[3] - x.name[3].mean()) ** 2).sum()) - 
           2 * x.name[-1] * ((x - x.mean()) * (x.name[3] - x.name[3].mean())).sum())
).reset_index()

<ipython-input-35-e5b5f6c52ac6>:1: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  w4 = w3.groupby(['SEC', 'ST_GR', 'no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat']).agg(
<ipython-input-35-e5b5f6c52ac6>:1: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  w4 = w3.groupby(['SEC', 'ST_GR', 'no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat']).agg(


In [36]:
# Calculate RSEs for each variable in T1
w5 = w4.assign(
    RSE_POP=lambda x: 100 * (x['var_pop'] ** 0.5) / (2 * x['pophat']),
    RSE_R1=lambda x: 100 * (x['mse_R1'] ** 0.5) / (2 * x['lfhat']),
    RSE_R2=lambda x: 100 * (x['mse_R2'] ** 0.5) / (2 * x['wrkhat'])
)

In [37]:
# assuming the data is loaded into a DataFrame called w5
T1 = w5[['SEC', 'ST_GR', 'no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2']].copy()

# add empty columns for sec_desc and st_gr_desc
T1['sec_desc'] = ''
T1['st_gr_desc'] = ''

# apply formatting to columns
T1['R1'] = 100 * T1['r1hat']
T1['R2'] = 100 * T1['r2hat']

# reorder columns
T1 = T1[['SEC', 'ST_GR', 'sec_desc', 'st_gr_desc', 'no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2']]

# print the resulting DataFrame
print(T1)

   SEC ST_GR sec_desc st_gr_desc  no_sam        pophat         lfhat  \
0    1    G1                       12324  2.016866e+08  6.203034e+07   
1    1    G2                        9517  3.310839e+07  1.236553e+07   
2    1    G3                       40407  5.329071e+08  1.969232e+08   
3    2    G1                        6700  4.284064e+07  1.408312e+07   
4    2    G2                        5947  5.803118e+06  2.087255e+06   
5    2    G3                       34961  2.717462e+08  9.700830e+07   

         wrkhat  RSE_POP         R1  RSE_R1         R2  RSE_R2  
0  5.776448e+07      0.0  30.755802     0.0  28.640710     0.0  
1  1.128034e+07      0.0  37.348616     0.0  34.070924     0.0  
2  1.831155e+08      0.0  36.952624     0.0  34.361612     0.0  
3  1.293553e+07      0.0  32.873269     0.0  30.194538     0.0  
4  1.881613e+06      0.0  35.967828     0.0  32.424176     0.0  
5  8.864419e+07      0.0  35.698120     0.0  32.620207     0.0  


In [38]:
T1 = w5[['SEC', 'ST_GR', 'no_sam', 'pophat','lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2']].copy()
T1['sec_desc'] = ' '*12
T1['st_gr_desc'] = ' '*12
T1['R1'] = 100 * T1['r1hat']
T1['R2'] = 100 * T1['r2hat']
T1 = T1[['SEC', 'ST_GR', 'no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2']]
T1

,SEC,ST_GR,no_sam,pophat,lfhat,wrkhat,RSE_POP,R1,RSE_R1,R2,RSE_R2
0,1,G1,12324,2.016866e+08,6.203034e+07,5.776448e+07,0.0,30.755802,0.0,28.640710,0.0
1,1,G2,9517,3.310839e+07,1.236553e+07,1.128034e+07,0.0,37.348616,0.0,34.070924,0.0
2,1,G3,40407,5.329071e+08,1.969232e+08,1.831155e+08,0.0,36.952624,0.0,34.361612,0.0
3,2,G1,6700,4.284064e+07,1.408312e+07,1.293553e+07,0.0,32.873269,0.0,30.194538,0.0
4,2,G2,5947,5.803118e+06,2.087255e+06,1.881613e+06,0.0,35.967828,0.0,32.424176,0.0
5,2,G3,34961,2.717462e+08,9.700830e+07,8.864419e+07,0.0,35.698120,0.0,32.620207,0.0


In [40]:
T1.to_csv('T1_Balanced_Half_Sample_Method.csv')